## Please be more open minded open exploiting efficient sparse lookups.

The `gurobipy.tupledict.prod` function really is great, but I honestly think you are being far too inefficient in how you exploit a dictish coefs argument. In fact, I find this inefficiency so far outside of "least astonishent" as to be a bug (albeit a minor one) but of course that is for you to decide. 

In [1]:
import gurobipy as gu

In [2]:
mdl = gu.Model()
super_vars = mdl.addVars([(1,i) for i in range(3,4000)] +  
                     [(2,i) for i in range(5,4000)] + 
                     [(3,i) for i in range(2,4000)], 
                     vtype=gu.GRB.BINARY,name='super_vars')
def yourSlicing():
    rtn = []
    for i,j in enumerate([1, 2, 3]):
        coefs = {(j, j*10): 10, (j, j*20):20, (j, j*30):30}
        rtn.append(super_vars.prod(coefs) <= i)
    return rtn

In [3]:
%timeit yourSlicing()

10 loops, best of 3: 69.7 ms per loop


In [4]:
mdl = gu.Model()
super_vars = mdl.addVars([(1,i) for i in range(3,4000)] +  
                     [(2,i) for i in range(5,4000)] + 
                     [(3,i) for i in range(2,4000)], 
                     vtype=gu.GRB.BINARY,name='super_vars')
def mySlicing():
    rtn = []
    for i,j in enumerate([1, 2, 3]):
        coefs = {(j, j*10): 10, (j, j*20):20, (j, j*30):30}
        rtn.append(gu.quicksum(v*super_vars[k] for k,v in coefs.items()) <= j)
    return rtn

In [5]:
%timeit mySlicing()

10000 loops, best of 3: 33.2 µs per loop


To my mind, the `mySlicing` look-up method is simply what I would expect `prod` to do when passed a dict. After all, `gurobipy.tupledict` is really a `dict`, and the pythonic expectation is that when two dict's cross product with each other they do it efficiently by exploitng the efficient hash-based indexing.